In [3]:
from utils.models import get_tokenizer, CustomModel
from utils.evaluation import evaluate_reward_model
from utils.data_preprocessing import CustomDataset, transform_df_for_dpo
from utils.loss_functions import RewardLoss
from utils.utils import get_reward
from utils.preference_generation import determine_preference

import torch
from torch.utils.data import DataLoader
from transformers import get_cosine_schedule_with_warmup

from tqdm import tqdm
import pandas as pd
import numpy as np
import gc, os
import glob

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [4]:
# --- Hyper‑parameters ---
wdir = '.'
MAX_LEN = 1024                         # truncate long GSM8K chains of thought
QUALITY = 'high'                        # Quality of Data to train on
OUTPUT_DIR = f"{wdir}/models/Reward_{QUALITY}"              # where to write LoRA adapter & tokenizer
BATCH_SIZE = 16
GRAD_ACCUM = 2                         # effective batch 32
LR = 1e-5
EPOCHS = 12
# Define total training steps
dataset_size = 10000
effective_batch_size = BATCH_SIZE * GRAD_ACCUM  # per_device_batch_size * num_gpus * grad_accum
TOTAL_STEP = (dataset_size // effective_batch_size + 1) * EPOCHS  # 684 steps
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Training Hyperparameters
eval_steps = GRAD_ACCUM * 20 # every 20 steps of update, evaluate
score_steps = GRAD_ACCUM * 200 # every 200 steps of update, calculate ground truth score.
early_stopping_patience = 3

prompt_length = 20
max_length = 196

rng = np.random.default_rng(42)
scale = 1.0

In [5]:

# Get Model
model_name = 'google/gemma-3-270m'

reward_model = CustomModel(model_name, 1).to(device)

tok = get_tokenizer(model_name)

In [6]:
# Path to the folder containing the CSV files
path = f'{wdir}/data/'

# Get a list of all CSV files in the folder
all_files = glob.glob(path + f"dpo_data_{QUALITY}.csv")

print(all_files)
# Expected output: ['data/sales_jan.csv', 'data/sales_feb.csv', 'data/sales_mar.csv']
# Read each CSV file into a DataFrame and store them in a list
list_of_dfs = [pd.read_csv(file) for file in all_files]

# Concatenate all DataFrames in the list by row
df = pd.concat(list_of_dfs, ignore_index=True)

df['pref'] = df.apply(determine_preference,
                      axis=1,
                      args=(scale, rng)  # scale=1.0, rng=custom generator)
)
print(df.pref.value_counts())

# # --- DATA PROCESS ---
df = transform_df_for_dpo(df)

['/content/drive/Othercomputers/My MacBook Pro/Oracle DPO/data/dpo_data_high.csv']
pref
1 > 2    6021
2 > 1    5979
Name: count, dtype: int64


In [7]:
# --- Create DataLoaders ---
train_dataset = CustomDataset(df[:dataset_size], torch.zeros(dataset_size), torch.zeros(dataset_size))
eval_dataset = CustomDataset(df[dataset_size:], torch.zeros(len(df) - dataset_size), torch.zeros(len(df) - dataset_size))
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda batch: batch)
eval_dataloader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, collate_fn=lambda batch: batch)

In [8]:
# --- Training Components ---
loss_fn = RewardLoss()
# The optimizer will only see the trainable PEFT parameters
optimizer = torch.optim.AdamW(
    reward_model.parameters(),
    lr=LR,
    betas=(0.9, 0.95),
    eps=1e-8,
    weight_decay=0.01
)


# Cosine scheduler
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps = int(0.03 * TOTAL_STEP),
    num_training_steps = TOTAL_STEP
)

In [ ]:
# --- Early Stopping and Model Saving Variables ---
best_eval_loss = float('inf')
patience_counter = 0
global_step = 0
best_eval_loss = evaluate_reward_model(reward_model, eval_dataloader, loss_fn, tok, device, max_length)
print(f"\nStep {global_step}: Validation Loss = {best_eval_loss:.4f}")
print("\n--- Starting Training ---")
for epoch in range(EPOCHS):

    # Note: optimizer.zero_grad() is now inside the accumulation block

    # Use enumerate to get the batch index 'i'
    for i, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")):
        reward_model.train()
        # --- Forward Pass ---
        r1 = get_reward(reward_model, tok, batch, 'first_responses', device, max_length)
        r2 = get_reward(reward_model, tok, batch, 'second_responses', device, max_length)

        # Stack the tensor items
        choices = torch.stack([item['choices'] for item in batch])

        loss = loss_fn(
            r1,
            r2,
            choices.to(device)
        )

        # --- Scale the Loss and Backpropagate ---
        loss = loss / GRAD_ACCUM
        loss.backward()

        # --- Optimizer Step ---
        if (global_step + 1) % GRAD_ACCUM == 0:
            torch.nn.utils.clip_grad_norm_(reward_model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        # --- Evaluation and Early Stopping Logic ---
        # This block should be inside the optimizer step block
        global_step += 1 # Increment global_step only when weights are updated
        if global_step % eval_steps == 0:
            eval_loss = evaluate_reward_model(reward_model, eval_dataloader, loss_fn, tok, device, max_length)
            print(f"\nStep {global_step}: Validation Loss = {eval_loss:.4f}")
            # # 2. Save a checkpoint at every evaluation step

            if eval_loss < best_eval_loss:
                print(f"\nValidation loss improved from {best_eval_loss} to {eval_loss}. Saving model...")
                best_model_dir = os.path.join(OUTPUT_DIR, "best_model")
                reward_model.save_pretrained(best_model_dir)
                best_eval_loss = eval_loss
                patience_counter = 0  # Reset patience
            else:
                patience_counter += 1
                print(f"\nValidation loss did not improve. Patience: {patience_counter}/{early_stopping_patience}")

            if patience_counter >= early_stopping_patience:
                print("\nEarly stopping triggered.")
                break
        gc.collect()
        torch.cuda.empty_cache()

    if patience_counter >= early_stopping_patience:
        break

print("\n--- Training Finished ---")


Step 0: Validation Loss = 1.1852

--- Starting Training ---


Epoch 1:   6%|▌         | 39/625 [00:32<07:57,  1.23it/s]


Step 40: Validation Loss = 1.1828

Validation loss improved from 1.185245222568512 to 1.182830538749695. Saving model...
Saving model weights to /content/drive/Othercomputers/My MacBook Pro/Oracle DPO/models/Reward_high/best_model


Epoch 1:  13%|█▎        | 79/625 [01:36<07:29,  1.21it/s]


Step 80: Validation Loss = 1.1738

Validation loss improved from 1.182830538749695 to 1.1737848720550537. Saving model...
Saving model weights to /content/drive/Othercomputers/My MacBook Pro/Oracle DPO/models/Reward_high/best_model


...


Step 3440: Validation Loss = 0.5517

Validation loss did not improve. Patience: 1/3


Epoch 6:  57%|█████▋    | 355/625 [09:38<46:43, 10.38s/it]


Step 3480: Validation Loss = 0.5543

Validation loss did not improve. Patience: 2/3


Epoch 6:  63%|██████▎   | 394/625 [10:43<06:17,  1.63s/it]


Step 3520: Validation Loss = 0.5556

Validation loss did not improve. Patience: 3/3

Early stopping triggered.

--- Training Finished ---
